In [ ]:
week1_raw = pd.read_csv("week1_results.csv")

week1_winners = week1_raw.loc[[0]]
week1_picks = week1_raw.copy()
week1_picks.drop([0, 1, 2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41], axis=0, inplace=True)

week1_picks.head()

df = week1_picks.pivot(index= , columns= , values= )